# Pancreas endocrinogenesis E1415 - Waddington OT

# Imports and settings

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import wot
import scanpy as sc
import scvelo as scv
import cellrank as cr
import anndata as ad
from cellrank.external.kernels import WOTKernel
from cellrank.estimators import GPCCA

In [ ]:
cr.logging.print_versions()

# Data loading and preprocessing

In [ ]:
adata825 = sc.read("/lustre/groups/ml01/workspace/monge_velo/data/benchmarks/gastrulation/stage_825_growth_rate.h5ad")
adata85 = sc.read("/lustre/groups/ml01/workspace/monge_velo/data/benchmarks/gastrulation/stage_85_growth_rate.h5ad")
adata = ad.concat([adata825, adata85], merge = 'same')

leave_in_cell = ['Blood progenitors 2', 'Erythroid1', 'Erythroid2', 'Erythroid3']
adata = adata[adata.obs["celltype"].isin(leave_in_cell)].copy()

sc.pp.log1p(adata)

adata.obs = adata.obs.loc[:, ['stage', 'celltype']]
adata.var = adata.var.loc[:, ['highly_variable']]
adata.obs['celltype'] = pd.Series(adata.obs['celltype'], dtype='category')

# Transition probabilities

In [ ]:
wk = WOTKernel(adata, time_key="stage")
wk.compute_initial_growth_rates(organism="mouse", key_added="growth_rate_init")
wk.compute_transition_matrix(
    growth_iters=3, growth_rate_key="growth_rate_init"
)

# Macrostate calculation

In [ ]:
g = GPCCA(wk)
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

In [ ]:
g.compute_macrostates(n_states=1, cluster_key="celltype")
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

# Fate probabilities

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        'Erythroid3'
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

In [ ]:
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=['Erythroid3'],
    cluster_key='celltype',
    clusters=['Blood progenitors 2', 'Erythroid1', 'Erythroid2', 'Erythroid3'],
    figsize=(17, 7),
    title="",
    #save="transprobs_wot.png"
)

In [ ]:
cluster = ['Blood progenitors 2', 'Erythroid1', 'Erythroid2', 'Erythroid3']
terminal_cluster = ['Erythroid3']

aggregated_absorption_prob = np.empty((len(terminal_cluster), len(cluster)))
data = g.absorption_probabilities.X
sums = np.sum(data, axis=0)
tmp = adata.obs['celltype']

for i in range(len(cluster)):
    aggregated_absorption_prob[:,i] = np.dot(np.array((tmp.array==cluster[i]).astype(int)),data)
for i in range(len(cluster)):
    aggregated_absorption_prob[:,i] = aggregated_absorption_prob[:,i]/np.sum(aggregated_absorption_prob[:,i])
df = pd.DataFrame(aggregated_absorption_prob)
df.index = terminal_cluster
df.columns = cluster

In [ ]:
correct_transitions = [('Erythroid3', 'Blood progenitors 2'), ('Erythroid3', 'Erythroid1'),
                       ('Erythroid3', 'Erythroid2'), ('Erythroid3', 'Erythroid3')]

sum_prob = 0
for i in correct_transitions:
    sum_prob = sum_prob + df.loc[i]
mean_prob = sum_prob/len(cluster)
mean_prob